In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from config import Config
import utils
import model as modellib
import visualize
from model import log

%matplotlib inline

Using TensorFlow backend.


In [2]:
# GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = "/scratch/wenyuan/Mask_RCNN_On_Pathology/Mask_RCNN/mask_rcnn_coco.h5"

In [44]:
import prostate
dataset_dir = "/scratch/wenyuan/Mask_RCNN_On_Pathology/Data_Pre_Processing/cedars-224"
held_out_set = 0
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
class InferenceConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    DETECTION_NMS_THRESHOLD = 0.7     
    MEAN_PIXEL = np.array(mean_pixel)
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

inference_config = InferenceConfig()
inference_config.display()
# Recreate the model in inference mode
model = modellib.ResNet_Classifier(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.7
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 193.95062641  120.83140887  183.74016669]
MINI_MASK_SHAPE                (56, 56)
NAME                           prostate
NUM_CLASSES                    4
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
PO

In [45]:
# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights
h5_filename = "logs/prostate20180413T1617-held-out-set0(partial)/mask_rcnn_prostate_0040.h5" 
# Specify the h5 filename here if you want to choose a specific file
epoch = -1

if h5_filename is not None:
    model_path = os.path.join(ROOT_DIR, h5_filename)
elif epoch == -1:    
    model_path = model.find_last()[1]
else:
    try:
        model_path = model.find_specific(epoch = epoch, held_out_set = held_out_set)[1]
    except:
        model_path = model.find_specific(epoch = epoch)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /scratch/wenyuan/Mask_RCNN_On_Pathology/ResNet/logs/prostate20180413T1617-held-out-set0(partial)/mask_rcnn_prostate_0040.h5


In [46]:
dataset_val = prostate.ProstateDataset()
_, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)
# val_list = [image for image in val_list if image not in exclude_list]
dataset_val.load_prostate(dataset_dir, val_list, mode = 16)
dataset_val.prepare()

In [47]:
gt_tumor_class = []
pred_tumor_class = []
num_wrong_prediction = 0
for image_id in dataset_val.image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    tumor_class_gt = 1 \
    if (sum(gt_class_id)) else 0
    
    # Run object detection
    results = model.detect([image], verbose=0)
    gt_tumor_class.append(tumor_class_gt)
    pred_tumor_class.append(results[0][1])
    if (np.argmax(results[0]) != tumor_class_gt):
        print(tumor_class_gt, results, image_id / 16, image_id % 16)
        num_wrong_prediction = num_wrong_prediction + 1

0 [[ 0.02673167  0.97326839]] 34.875 14
0 [[  2.00949066e-12   1.00000000e+00]] 34.9375 15
0 [[ 0.00241592  0.9975841 ]] 36.25 4
0 [[  7.56522932e-04   9.99243498e-01]] 36.3125 5
0 [[ 0.02651601  0.97348398]] 36.375 6
0 [[  4.89959857e-05   9.99951005e-01]] 36.5 8
0 [[  1.12337917e-10   1.00000000e+00]] 50.25 4
0 [[ 0.26919314  0.73080683]] 58.8125 13
1 [[ 0.99888974  0.00111032]] 59.875 14
0 [[  1.77786642e-04   9.99822199e-01]] 59.9375 15
0 [[ 0.05355411  0.94644588]] 79.125 2
0 [[ 0.2006488   0.79935122]] 79.1875 3
0 [[ 0.20439368  0.79560632]] 79.4375 7
0 [[  1.60756113e-11   1.00000000e+00]] 79.625 10
0 [[  7.95017113e-05   9.99920487e-01]] 79.6875 11
0 [[ 0.02498595  0.97501409]] 79.9375 15
0 [[  8.34215363e-10   1.00000000e+00]] 81.9375 15
1 [[  1.00000000e+00   1.14075061e-09]] 85.875 14
1 [[  9.99864936e-01   1.35097594e-04]] 85.9375 15
1 [[ 0.93465167  0.06534837]] 95.5 8
0 [[ 0.10071664  0.89928341]] 97.0 0
0 [[ 0.02771444  0.97228563]] 97.125 2
0 [[ 0.04850636  0.95149368]]

In [49]:
# from sklearn import metrics
# fpr, tpr, thresholds = metrics.roc_curve(gt_tumor_class, pred_tumor_class)

In [50]:
# roc_auc = metrics.auc(fpr, tpr)

In [51]:
# plt.figure()
# lw = 2
# plt.plot(fpr, tpr, color='darkorange',
#          lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()

In [52]:
## Save results:
# result_dict = {"fpr":fpr.tolist(), "tpr":tpr.tolist()}
# filepath = 'ROC_held_out_%d.csv'%held_out_set

In [53]:
# import csv
# with open(filepath, "w") as outfile:
#    writer = csv.writer(outfile)
#    writer.writerow(result_dict.keys())
#    writer.writerows(zip(*result_dict.values()))